In [2]:
import os

path = '/Users/matthewhwang/Galvanize/fightclub/data/raw/Left Jab'

folders = []

# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for folder in d:
        folders.append(os.path.join(r, folder))

for f in folders:
    print(f)

/Users/matthewhwang/Galvanize/fightclub/data/raw/Left Jab/Left Jab 100 Matt Combo RC_2019-11-05T20-19-06.646
/Users/matthewhwang/Galvanize/fightclub/data/raw/Left Jab/Left Jab 10 Jack_2019-11-05T16-19-57.817
/Users/matthewhwang/Galvanize/fightclub/data/raw/Left Jab/Left Jab 50 Egor Combo RH_2019-11-05T21-06-15.179
/Users/matthewhwang/Galvanize/fightclub/data/raw/Left Jab/Left Jab 200 Matt Combo RC_2019-11-09T14-06-59.036
/Users/matthewhwang/Galvanize/fightclub/data/raw/Left Jab/Left Jab 25 Egor_2019-11-05T16-24-17.326
/Users/matthewhwang/Galvanize/fightclub/data/raw/Left Jab/Left Jab 50 Matt_2019-11-05T14-14-39.859
/Users/matthewhwang/Galvanize/fightclub/data/raw/Left Jab/Left Jab 100 Egor Combo RC_2019-11-05T20-25-14.955
/Users/matthewhwang/Galvanize/fightclub/data/raw/Left Jab/Left Jab 20 Nathan_2019-11-06T22-02-50.040
/Users/matthewhwang/Galvanize/fightclub/data/raw/Left Jab/Left Jab 25 David_2019-11-06T22-21-13.775
/Users/matthewhwang/Galvanize/fightclub/data/raw/Left Jab/Left Jab 

In [4]:
import glob

path = 'c:\\projects\\hc2\\'

files = [f for f in glob.glob(path + "**/*.txt", recursive=True)]

for f in files:
    print(f)

In [20]:
files = [f for f in glob.glob(folders[0] + "**/*.csv")]

In [21]:
files

['/Users/matthewhwang/Galvanize/fightclub/data/raw/Left Jab/Left Jab 100 Matt Combo RC_2019-11-05T20-19-06.646/Left Jab 100 Matt Combo RC_Accelerometer_2019-11-05T20-19-06.646.csv',
 '/Users/matthewhwang/Galvanize/fightclub/data/raw/Left Jab/Left Jab 100 Matt Combo RC_2019-11-05T20-19-06.646/Left Jab 100 Matt Combo RC_Gyroscope_2019-11-05T20-19-06.646.csv']

In [14]:
folders[0]

'/Users/matthewhwang/Galvanize/fightclub/data/raw/Left Jab/Left Jab 100 Matt Combo RC_2019-11-05T20-19-06.646'

In [19]:
files[0]

'/Users/matthewhwang/Galvanize/fightclub/data/raw/Left Jab/Left Jab 100 Matt Combo RC_2019-11-05T20-19-06.646/Left Jab 100 Matt Combo RC_Accelerometer_2019-11-05T20-19-06.646.csv'

In [22]:
import pandas as pd

In [23]:
accel = pd.read_csv(files[0])

In [24]:
accel.head()

,epoch (ms),time (-08:00),elapsed (s),X-Axis (g),Y-Axis (g),Z-Axis (g)
0,1573013946727,2019-11-05T20:19:06.727,0.000,-0.781,0.036,0.689
1,1573013946766,2019-11-05T20:19:06.766,0.039,-0.789,0.039,0.695
2,1573013946807,2019-11-05T20:19:06.807,0.080,-0.789,0.044,0.694
3,1573013946847,2019-11-05T20:19:06.847,0.120,-0.784,0.057,0.685
4,1573013946888,2019-11-05T20:19:06.888,0.161,-0.785,0.055,0.692


In [29]:
name = files[0].split('/')[-1].split(' ')

In [30]:
name

['Left',
 'Jab',
 '100',
 'Matt',
 'Combo',
 'RC_Accelerometer_2019-11-05T20-19-06.646.csv']

In [31]:
category = name[0] + " " + name[1]

In [32]:
category

'Left Jab'

In [35]:
count = int(name[2])

In [36]:
count

100